In [1]:
import customtkinter as ctk
import json
import traceback

entry_count = 1

json_data = {}

table_grid_buttons: list[ctk.CTkButton] = [] 

delete_buttons: list[ctk.CTkButton] = [] 

total_selected_buttons: list[ctk.CTkButton] = []     # Combines all buttons in a list --> button list

collected_buttons: list[ctk.CTkButton] = []         # Combines only currently collected buttons in a list --> button list

widgets: dict = {}



def show_cursor(event):
    input_text_field.configure(state="normal")


def press_return_event(event):
    collect_button.invoke()



def collect_button_click():
    global entry_count
    entry_key = f"entry_{entry_count}"
    collected_entities = []

    if collected_buttons:
        for collected_button in collected_buttons:
            collected_entities.append(collected_button.cget("text"))
            collected_button.configure(state="disabled")
    
        json_data[entry_key] = collected_entities

        if not json_data[entry_key]:
            del json_data[entry_key]
        
        collected_buttons.clear()
        entry_count += 1
    print("enter pressed")



def create_entry_for_button_list(cell_button: ctk.CTkButton):
    total_selected_buttons.append(cell_button)
    collected_buttons.append(cell_button)


def delete_last_entry_for_button_list(cell_button: ctk.CTkButton):
    total_selected_buttons.remove(cell_button)
    collected_buttons.remove(cell_button)


def grid_button_click(cell_button: ctk.CTkButton, cell_value: str|int|None):
    if cell_button.cget("fg_color") == "#3B8ED0":
        cell_button.configure(fg_color="#1F6AA5")
        create_entry_for_button_list(cell_button)
    else:
        cell_button.configure(fg_color="#3B8ED0")
        delete_last_entry_for_button_list(cell_button)
    input_text_field.configure(state="disabled")    # Deactivate cursor in textfield so pressing enter doesnt move the cursor
    print(collected_buttons)
    

def export_json():
    global entry_count
    for button in total_selected_buttons:
        button.configure(state="normal", fg_color="#3B8ED0")    # Return color to light blue
    print(json_data)
    json_data.clear()
    entry_count = 1


def generate_table():
    input_text = input_text_field.get("1.0", "end-1c")  # Get the text from the input field
    # Clear all button lists when table is generated
    clear_buttons()
    try:
        # Parse the input text as a list of lists
        data = eval(input_text)
        create_table(data)
    except Exception as e:
        traceback.print_exc()
        print(f"Error parsing input: {e}")


def cell_clicked(row, col):
    print(f"Cell clicked: Row {row}, Column {col}")  # Replace with your logic (e.g., JSON population)


def height_maximizer(tracker, height):
    if tracker < height:
        tracker = height
    else:
        tracker = tracker
    return tracker


def set_max_height_for_table_grid_buttons(button_list: list[ctk.CTkButton], tracker: int):
    for button in button_list:
        button.configure(height=tracker)


def update_button_height_tracker(button: ctk.CTkButton, tracker: int):
    """
    
    """
    root.update_idletasks()
    current_height = button.winfo_height()
    button_height_tracker = height_maximizer(tracker, current_height)
    return button_height_tracker


def destroy_widgets(*widgets: list[ctk.CTkButton]):
    for button_list in widgets:
        for button in button_list:
            button.destroy()


def clear_buttons():
    destroy_widgets(table_grid_buttons, delete_buttons)
    table_grid_buttons.clear()
    delete_buttons.clear()
    total_selected_buttons.clear()
    collected_buttons.clear()



def create_table(data):

    # Create an inner frame for the clickable grid
    if "inner_frame" in widgets:
        del widgets["inner_frame"]
    widgets["inner_frame"] = ctk.CTkFrame(master=outer_frame, corner_radius=0, bg_color="black", fg_color="#DBDBDB")
    widgets["inner_frame"].grid(row=2, column=0, padx=10, pady=10)

    # Running Parameters
    button_height_tracker = 0   # Tracks pixel height of largest cell
    
    # Get row and column size of table
    rows = len(data)
    cols = len(data[0]) if rows > 0 else 0  # Addition of "1" for tick box

    # parse through table grid
    for row in range(rows):
        for col in range(cols):
            cell_value = data[row][col]
            cell_button = ctk.CTkButton(
                master=widgets["inner_frame"], 
                text=cell_value, 
                corner_radius=0, 
                width=140, 
                border_spacing=0,
                fg_color="#3B8ED0",
                border_width=1,
                border_color="black"
            )   # Create button for every cell
            
            # Organize buttons in grid and create JSON entry when clicked
            cell_button.grid(row=row, column=col, padx=0, pady=0)
            cell_button.configure(command=lambda c = cell_value, b = cell_button: grid_button_click(b, c))

            # Configure line break if words exceed 120 pixels
            if len(str(cell_value)) != 0:
                cell_button._text_label.configure(wraplength=120)

            # Create special design for column headers
            if row == 0:
                cell_button.configure(font=('Arial', 14, 'bold'), fg_color="grey") # Use inner_frame as the parent
            
            # Update button_height_tracker to maximize button height
            button_height_tracker = update_button_height_tracker(cell_button, button_height_tracker) #stores highest height
            
            # Collect all button objects in list to apply tracked max height after loop is over
            table_grid_buttons.append(cell_button)

        # Delete Button --> delete one collection      
        delete_button = ctk.CTkButton(
            master=widgets["inner_frame"],
            text="X",
            text_color="#FAE3DC",
            font=('Arial', 14, 'bold'),
            width=20,
            height=20,
            corner_radius=8,
            fg_color="#E05B4F",  # White text on red background
            hover_color=("darkred", "darkred"),
            )
        delete_button.grid(row=row, column=cols, padx=5, pady=0)
        delete_buttons.append(delete_button)
    
    # Set highest found height for all buttons
    set_max_height_for_table_grid_buttons(table_grid_buttons, button_height_tracker)
    
    # Activate Export JSON button
    print_json_button.configure(state="normal") # Activate button after table is created
    collect_button.configure(state="normal")
    input_text_field.configure(state="disabled")



if __name__ == "__main__":
    root = ctk.CTk()
    root.title("Clickable Table Generator")

    # Create an outer frame
    outer_frame = ctk.CTkFrame(master=root)
    outer_frame.pack()

    # Text field for input
    input_text_field = ctk.CTkTextbox(master=outer_frame, width=800)
    input_text_field.grid(row=0, column=0, padx=10, pady=10)
    input_text_field.bind("<Button-1>", show_cursor)

    # Button to generate the table
    generate_button = ctk.CTkButton(master=outer_frame, text="Generate Table", command=generate_table)
    generate_button.grid(row=1, column=0, padx=10, pady=10)

    # Button frame for collect and export button
    process_button_frame = ctk.CTkFrame(master=outer_frame)
    process_button_frame.grid(row=4, column=0, padx=10, pady=10)

    # Button to add a list with entries to json_dict
    collect_button = ctk.CTkButton(master=process_button_frame, text="Collect (Press Enter)", command=collect_button_click, state="disabled")
    collect_button.grid(row=0, column=0, padx=10, pady=10)
    root.bind("<Return>", press_return_event)   # Button can be pressed via Return key

    # Button to generate the JSON file
    print_json_button = ctk.CTkButton(master=process_button_frame, text="Print JSON", command=export_json, state="disabled")
    print_json_button.grid(row=0, column=1, padx=10, pady=10)
    print(dir(print_json_button))
 

    root.mainloop()


['_CTkAppearanceModeBaseClass__appearance_mode', '_CTkScalingBaseClass__scaling_type', '_CTkScalingBaseClass__widget_scaling', '_Misc__winfo_getint', '_Misc__winfo_parseitem', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_anchor', '_apply_appearance_mode', '_apply_argument_scaling', '_apply_font_scaling', '_apply_geometry_scaling', '_apply_widget_scaling', '_apply_window_scaling', '_background_corner_colors', '_bg_color', '_bind', '_border_color', '_border_spacing', '_border_width', '_canvas', '_check_color_type', '_check_font_type', '_check_image_type', '_click_animation', '_click_animation_running', '_clicked', '_command', '_

In [ ]:
[[1, "Ageakjfhasjdasjdlajsdlkjas", "Country"], ["aasdasdagsdgsgsfsfsdfsdfsdfsghdfhdfgdsgdgdfgdfgdfgdfgdfgdfgdfg", "25", "USA"], ["Bob", "30asdasdasdasdasd", "dfgdgdfgdfgdfgdfgdfgdfgdfgdfgdfgdgfgfgfgfgdgdgfgdgfgdg"], ["Charlie", "22", "UK"]]

In [7]:
import customtkinter as ctk

# Assuming 'ctk' is imported from somewhere else in your code

def on_checkbox_click(row):
    # Add your logic for checkbox click here
    print(f"Checkbox clicked for row {row}")

# Assuming 'data' is your 2D array of cell values
data = [
    ["A1", "B1", "C1"],
    ["A2", "B2", "C2"],
    # ... add more rows as needed
]

root = ctk.CTk()
root.title("Table with Checkboxes")

# Assuming 'inner_frame' is your inner frame widget
inner_frame = ctk.CTkFrame(root)
inner_frame.grid(row=0, column=0, padx=10, pady=10)

rows = len(data)
cols = len(data[0]) if rows > 0 else 0  # Addition of "1" for checkbox

# parse through table grid
for row in range(rows):
    for col in range(cols):
        cell_value = data[row][col]
        cell_button = ctk.CTkButton(
            master=inner_frame,
            text=cell_value,
            corner_radius=0,
            width=140,
            border_spacing=5,
            fg_color="#3B8ED0"
        )  # Create button for every cell

        cell_button.grid(row=row, column=col, padx=5, pady=5)

    # Add a checkbox at the end of each row
    checkbox = ctk.CTkButton(
        master=inner_frame,
        command=lambda r=row: on_checkbox_click(r),
        text="X",
        text_color="#FAE3DC",
        font=('Arial', 18, 'bold'),
        width=28,
        height=28,
        corner_radius=8,
        fg_color="#E05B4F",  # White text on red background
        hover_color=("darkred", "darkred"),
        )
    checkbox.grid(row=row, column=cols, padx=5, pady=5)

root.mainloop()
#E05B4F


In [10]:
from tkinter import *
import datetime

root = Tk()
lab = Label(root)
lab.pack()

def update_clock():
    current_time = datetime.datetime.now().strftime("Time: %H:%M:%S")
    lab.config(text=current_time)
    root.after(1000, update_clock)  # Run itself again after 1000 ms

update_clock()
root.mainloop()

In [28]:
dict_ = {"bruh":[]}

list_ = [1,2,3,4,5]

dict_["bruh"].extend(list_)

In [35]:
import customtkinter as ctk
from customtkinter import CTkButton
from collections import defaultdict

root1 = ctk.CTk()

test_dict = defaultdict(list)

var1 = "moin"

frame1 = ctk.CTkFrame(master=root1)
frame1.pack()
test_dict["master"] = frame1
frame_name1 = frame1.winfo_name()


button1 = ctk.CTkButton(master=frame1, text=var1)
button1.pack()
test_dict["buttons"].append(button1)

def test_func(button: CTkButton, *params):
    print(params)

    for param, value in params:
        print(param, value)
        button.configure(param=value)

param = ("fg_color", "black")
test_func(button1, param)


button2 = ctk.CTkButton(master=frame1, text=var1)
button2.pack()
test_dict["buttons"].append(button2)

print(test_dict)

print(root1)
root1.mainloop()

def destroy_children(x):
    return test_dict["master"].children[x].destroy()

#for x in test_dict["buttons"]:
#   test_dict["master"].children[x].destroy()



print(test_dict)

(('fg_color', 'black'),)
fg_color black


ValueError: ['param'] are not supported arguments. Look at the documentation for supported arguments.

In [ ]:
#TESTGELÄNDE

import customtkinter as ctk
from customtkinter import CTkButton, CTkFrame
import json
import traceback

"""
Global Vars and constants
"""

entry_count = 1

MASTER =                    "master"
BUTTONS =                   "buttons"
TABLE_GRID_BUTTONS =        "table_grid_buttons"
TOTAL_SELECTED_BUTTONS =    "total_selected_buttons"
COLLECTED_BUTTONS =         "collected_buttons"
DELETE_BUTTONS =            "delete_buttons"

"""
Global Vars
---------------------------------------------
"""


json_data: dict[CTkButton, str] = {}


"""
Button Trackers
"""

button_tracker: dict[str, CTkFrame|None|dict[str, list]] = {
    MASTER:None, 
    BUTTONS:
    {
        TABLE_GRID_BUTTONS:[],     # all buttons from table grid
        TOTAL_SELECTED_BUTTONS:[], # buttons that are selected (held down)
        COLLECTED_BUTTONS:[],      # buttons that were collected
        DELETE_BUTTONS:[]          # red buttons to delete a line
    }   
}                                                                          

"""
Button Trackers
---------------------------------------------
"""



"""
Utilities
"""
def clear_buttons(button_tracker: dict[str, list[str]], key: str):
    button_tracker[key].clear()


def destroy_widgets(button_tracker: dict[str, CTkFrame]):
    button_tracker[MASTER].destroy()
    button_tracker[MASTER] = None

"""
Utilities
---------------------------------------------
"""

widgets: dict = {}



def show_cursor(event):
    input_text_field.configure(state="normal")


def press_return_event(event):
    collect_button.invoke()



def collect_button_click():
    global entry_count
    entry_key = f"entry_{entry_count}"
    collected_entities = []

    if collected_buttons:
        for collected_button in collected_buttons:
            collected_entities.append(collected_button.cget("text"))
            collected_button.configure(state="disabled")
    
        json_data[entry_key] = collected_entities

        if not json_data[entry_key]:
            del json_data[entry_key]
        
        collected_buttons.clear()
        entry_count += 1
    print("enter pressed")



def create_entry_for_button_list(button_name: str):
    button_tracker[BUTTONS][TOTAL_SELECTED_BUTTONS].append(button_name)
    button_tracker[BUTTONS][COLLECTED_BUTTONS].append(button_name)


def delete_last_entry_for_button_list(button_name: str):
    button_tracker[BUTTONS][TOTAL_SELECTED_BUTTONS].remove(button_name)
    button_tracker[BUTTONS][COLLECTED_BUTTONS].remove(button_name)


def request_grid_button_param(button_name: str, property: str):
    return button_tracker[MASTER].children[button_name].cget(property)


def configure_grid_button_param(button_name: str, *params: tuple[str,any]):
    for param, value in params:
        button_tracker[MASTER].children[button_name].configure(**{param:value})


def grid_button_click(button_name: str, cell_value: str|int|None):
    if request_grid_button_param(button_name, "fg_color") == "#3B8ED0":
        color_dark_blue = ("fg_color", "#1F6AA5")
        configure_grid_button_param(button_name, color_dark_blue)
        create_entry_for_button_list(button_name)
    else:
        color_light_blue = ("fg_color", "#3B8ED0")
        configure_grid_button_param(button_name, color_light_blue)
        delete_last_entry_for_button_list(button_name)
    input_text_field.configure(state="disabled")    # Deactivate cursor in textfield so pressing enter doesnt move the cursor
    print(collected_buttons)
    

def export_json():
    global entry_count
    for button in total_selected_buttons:
        button.configure(state="normal", fg_color="#3B8ED0")    # Return color to light blue
    print(json_data)
    json_data.clear()
    entry_count = 1


def generate_table():
    global button_tracker
    input_text = input_text_field.get("1.0", "end-1c")  # Get the text from the input field
    # Destroy button widgets and Clear all button lists when table is generated
    destroy_widgets(button_tracker)
    for button_list in button_tracker[BUTTONS].values():
        button_list.clear()
    try:
        # Parse the input text as a list of lists
        data = eval(input_text)
        create_table(data)
    except Exception as e:
        traceback.print_exc()
        print(f"Error parsing input: {e}")


def cell_clicked(row, col):
    print(f"Cell clicked: Row {row}, Column {col}")  # Replace with your logic (e.g., JSON population)


def height_maximizer(tracker, height):
    if tracker < height:
        tracker = height
    else:
        tracker = tracker
    return tracker


def set_max_height_for_table_grid_buttons(button_list: list[ctk.CTkButton], tracker: int):
    for button in button_list:
        button.configure(height=tracker)


def update_button_height_tracker(button: ctk.CTkButton, tracker: int):
    """
    """
    root.update_idletasks()
    current_height = button.winfo_height()
    button_height_tracker = height_maximizer(tracker, current_height)
    return button_height_tracker



def create_button_frame(master_):
    # Create an inner frame for the clickable grid
    button_tracker[MASTER] = ctk.CTkFrame(master=master_, corner_radius=0, bg_color="black", fg_color="#DBDBDB")
    button_tracker[MASTER].grid(row=2, column=0, padx=10, pady=10)



def create_table(data):

    create_button_frame(master_=outer_frame)

    # Running Parameters
    button_height_tracker = 0   # Tracks pixel height of largest cell
    
    # Get row and column size of table
    rows = len(data)
    cols = len(data[0]) if rows > 0 else 0  # Addition of "1" for tick box

    # parse through table grid
    for row in range(rows):
        for col in range(cols):
            cell_value = data[row][col]
            cell_button = ctk.CTkButton(
                master=button_tracker[MASTER], 
                text=cell_value, 
                corner_radius=0, 
                width=140, 
                border_spacing=0,
                fg_color="#3B8ED0",
                border_width=1,
                border_color="black"
            )   # Create button for every cell
            
            # Organize buttons in grid and create JSON entry when clicked
            cell_button.grid(row=row, column=col, padx=0, pady=0)
            button_name = cell_button.winfo_name()
            cell_button.configure(command=lambda c = cell_value, b = button_name: grid_button_click(b, c))

            # Configure line break if words exceed 120 pixels
            if len(str(cell_value)) != 0:
                cell_button._text_label.configure(wraplength=120)

            # Create special design for column headers
            if row == 0:
                cell_button.configure(font=('Arial', 14, 'bold'), fg_color="grey") # Use inner_frame as the parent
            
            # Update button_height_tracker to maximize button height
            button_height_tracker = update_button_height_tracker(cell_button, button_height_tracker) #stores highest height
            
            # Collect all button objects in list to apply tracked max height after loop is over
            button_tracker[BUTTONS][TABLE_GRID_BUTTONS].append(cell_button)

        # Delete Button --> delete one collection      
        delete_button = ctk.CTkButton(
            master=widgets["inner_frame"],
            text="X",
            text_color="#FAE3DC",
            font=('Arial', 14, 'bold'),
            width=20,
            height=20,
            corner_radius=8,
            fg_color="#E05B4F",  # White text on red background
            hover_color=("darkred", "darkred"),
            )
        delete_button.grid(row=row, column=cols, padx=5, pady=0)
        button_tracker[BUTTONS][DELETE_BUTTONS].append(delete_button)
        delete_buttons.append(delete_button)
    
    # Set highest found height for all buttons
    set_max_height_for_table_grid_buttons(table_grid_buttons, button_height_tracker)
    
    # Activate Export JSON button
    print_json_button.configure(state="normal") # Activate button after table is created
    collect_button.configure(state="normal")
    input_text_field.configure(state="disabled")



if __name__ == "__main__":
    root = ctk.CTk()
    root.title("Clickable Table Generator")

    # Create an outer frame
    outer_frame = ctk.CTkFrame(master=root)
    outer_frame.pack()

    # Text field for input
    input_text_field = ctk.CTkTextbox(master=outer_frame, width=800)
    input_text_field.grid(row=0, column=0, padx=10, pady=10)
    input_text_field.bind("<Button-1>", show_cursor)

    # Button to generate the table
    generate_button = ctk.CTkButton(master=outer_frame, text="Generate Table", command=generate_table)
    generate_button.grid(row=1, column=0, padx=10, pady=10)

    # Button frame for collect and export button
    process_button_frame = ctk.CTkFrame(master=outer_frame)
    process_button_frame.grid(row=4, column=0, padx=10, pady=10)

    # Button to add a list with entries to json_dict
    collect_button = ctk.CTkButton(master=process_button_frame, text="Collect (Press Enter)", command=collect_button_click, state="disabled")
    collect_button.grid(row=0, column=0, padx=10, pady=10)
    root.bind("<Return>", press_return_event)   # Button can be pressed via Return key

    # Button to generate the JSON file
    print_json_button = ctk.CTkButton(master=process_button_frame, text="Print JSON", command=export_json, state="disabled")
    print_json_button.grid(row=0, column=1, padx=10, pady=10)
    print(dir(print_json_button))


    root.mainloop()

In [15]:
var1 = str()

dictionary = {}

dictionary[var1] = "string1"

var1 = int()

print(dictionary)

{'': 'string1'}


In [16]:
# Using a list
var1 = ["string1"]
my_dict = {"key1": var1}

# Now, var1 points to the same list object
print(my_dict["key1"])  # Output: ['string1']

# Modify var1 and observe the change in the dictionary
var1.append("new_string")
print(my_dict["key1"])  # Output: ['string1', 'new_string']

['string1']
['string1', 'new_string']


In [13]:
from customtkinter import CTkButton, CTkFrame

entry_count = 1

MASTER =                    "master"
BUTTONS =                   "buttons"
TABLE_GRID_BUTTONS =        "table_grid_buttons"
TOTAL_SELECTED_BUTTONS =    "total_selected_buttons"
COLLECTED_BUTTONS =         "collected_buttons"
DELETE_BUTTONS =            "delete_buttons"

"""
Global Vars
---------------------------------------------
"""


json_data: dict[CTkButton, str] = {}


"""
Button Trackers
"""

button_tracker: dict[str, CTkFrame|list[str]] = {
    MASTER:None, 
    TABLE_GRID_BUTTONS:[1],     # all buttons from table grid
    TOTAL_SELECTED_BUTTONS:[2], # buttons that are selected (held down)
    COLLECTED_BUTTONS:[3],      # buttons that were collected
    DELETE_BUTTONS:[4]          # red buttons to delete a line  
}   

print(button_tracker)

for key, value in button_tracker.items():
    if isinstance(value, list):
       button_tracker[key].clear()

print(button_tracker)

{'master': None, 'table_grid_buttons': [1], 'total_selected_buttons': [2], 'collected_buttons': [3], 'delete_buttons': [4]}
{'master': None, 'table_grid_buttons': [], 'total_selected_buttons': [], 'collected_buttons': [], 'delete_buttons': []}


In [17]:
str_ = "asd"

if str_:
    print("something")

something


In [21]:
tuple_ = (1,2,3,4)

print(*tuple_)

1 2 3 4
